#Nifty 50 Companies Tickers

In [1]:
import pandas as pd
nifty50_url = 'https://www1.nseindia.com/content/indices/ind_nifty50list.csv'
df_n50 = pd.read_csv(nifty50_url)
companies = df_n50['Company Name'].tolist()
for i in range(len(companies)):
  comp=companies[i].split(' Ltd')
  companies[i]=comp[0].strip()
  print(companies[i])

Adani Enterprises
Adani Ports and Special Economic Zone
Apollo Hospitals Enterprise
Asian Paints
Axis Bank
Bajaj Auto
Bajaj Finance
Bajaj Finserv
Bharat Petroleum Corporation
Bharti Airtel
Britannia Industries
Cipla
Coal India
Divi's Laboratories
Dr. Reddy's Laboratories
Eicher Motors
Grasim Industries
HCL Technologies
HDFC Bank
HDFC Life Insurance Company
Hero MotoCorp
Hindalco Industries
Hindustan Unilever
Housing Development Finance Corporation
ICICI Bank
ITC
IndusInd Bank
Infosys
JSW Steel
Kotak Mahindra Bank
Larsen & Toubro
Mahindra & Mahindra
Maruti Suzuki India
NTPC
Nestle India
Oil & Natural Gas Corporation
Power Grid Corporation of India
Reliance Industries
SBI Life Insurance Company
State Bank of India
Sun Pharmaceutical Industries
Tata Consultancy Services
Tata Consumer Products
Tata Motors
Tata Steel
Tech Mahindra
Titan Company
UPL
UltraTech Cement
Wipro


##Sentiment Analysis from Twitter

In [2]:
pip install tweepy


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: joblib in c:\python\python39\lib\site-packages (from nltk>=3.1->textblob) (1.1.0)



You should consider upgrading via the 'c:\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
market_terms="""10-K
EPS
Book Value
Market Value
Share price
Tax
Sales
Expenditure
EPS
ROE
Net Profit
Liabilities
Assets
IPO
Deal
Crash
Fair Value
Cash Flow
Gross Profit
Investment
Margin
Income
Portfolio
Revenues
Share Price"""
market_terms=list(set(list(market_terms.split('\n'))))
print(market_terms)
print(len(market_terms))

['10-K', 'Portfolio', 'Margin', 'Revenues', 'Share price', 'Expenditure', 'Income', 'Net Profit', 'Liabilities', 'Investment', 'Assets', 'Gross Profit', 'Cash Flow', 'ROE', 'Tax', 'Deal', 'Fair Value', 'Book Value', 'Sales', 'Crash', 'Market Value', 'IPO', 'Share Price', 'EPS']
24


In [5]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
sentimentlist = []
class TwitterClient(object):
	'''
	Generic Twitter Class for sentiment analysis.
	'''
	def __init__(self):
		'''
		Class constructor or initialization method.
		'''
		# keys and tokens from the Twitter Dev Console
		consumer_key = 'wqvwcbFpMOw8aT9llFwCRzseg'
		consumer_secret = 'iecws8RhMhTvhbN2ugqwbRuBUygMBu05QtcQz2MRGOiIoztRee'
		access_token = '1537785639529287682-I2KMSyeG1WGPULMdhA5Ncl6ySBnKIn'
		access_token_secret = 'MeFcK2pdZXneTRg8G5vLq2ty95PFbImyQqugNiATCgZ8g'

		# attempt authentication
		try:
			# create OAuthHandler object
			self.auth = OAuthHandler(consumer_key, consumer_secret)
			# set access token and secret
			self.auth.set_access_token(access_token, access_token_secret)
			# create tweepy API object to fetch tweets
			self.api = tweepy.API(self.auth)
		except:
			print("Error: Authentication Failed")

	def clean_tweet(self, tweet):
		'''
		Utility function to clean tweet text by removing links, special characters
		using simple regex statements.
		'''
		return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

	def get_tweet_sentiment(self, tweet):
		'''
		Utility function to classify sentiment of passed tweet
		using textblob's sentiment method
		'''
		# create TextBlob object of passed tweet text
		analysis = TextBlob(self.clean_tweet(tweet))
		# set sentiment
		if analysis.sentiment.polarity > 0:
			return 'positive'
		elif analysis.sentiment.polarity == 0:
			return 'neutral'
		else:
			return 'negative'

	def get_tweets(self, query, count = 100):
		'''
		Main function to fetch tweets and parse them.
		'''
		# empty list to store parsed tweets
		tweets = []

		try:
			# call twitter api to fetch tweets
			fetched_tweets = self.api.search_tweets(q = query, count = count)

			# parsing tweets one by one
			for tweet in fetched_tweets:
				# empty dictionary to store required params of a tweet
				parsed_tweet = {}

				# saving text of tweet
				parsed_tweet['text'] = tweet.text
				# saving sentiment of tweet
				parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

				# appending parsed tweet to tweets list
				if tweet.retweet_count > 0:
					# if tweet has retweets, ensure that it is appended only once
					if parsed_tweet not in tweets:
						tweets.append(parsed_tweet)
				else:
					tweets.append(parsed_tweet)

			# return parsed tweets
			return tweets

		except tweepy.TweepError as e:
			# print error (if any)
			print("Error : " + str(e))

def main():
	api=TwitterClient()
	tweets=dict()
	#print(' '.join(market_terms))
	#print(len(' '.join(market_terms).split()))
	market_tweets=api.get_tweets(query = 'indian market OR growth OR investment OR crash OR infrastructure OR globalisation OR IPO OR revenue OR TAX OR Duties OR RBI OR Stock Market OR NSE OR NSE OR Market Recession OR Market Inflation ', count = 500)
	#print(market_tweets)
	overallptweets=[tweet for tweet in market_tweets if tweet['sentiment'] == 'positive']
	overallntweets=[tweet for tweet in market_tweets if tweet['sentiment'] == 'negative']
	print("Market Negative tweets percentage: {} %".format(100*len(overallntweets)/len(market_tweets)))
	print("Market Positive tweets percentage: {} %".format(100*len(overallptweets)/len(market_tweets)))
	for company in companies:
		print("\n\nCompany - ", company)
		Query=str(company)#+' '  + ' '.join(market_terms)
		print(Query)
		tweets[company] = api.get_tweets(query = Query, count = 500)
    #totaltweets+=100
    #print(tweets[company])
    
    # picking positive tweets from tweets
		ptweets = [tweet for tweet in tweets[company] if tweet['sentiment'] == 'positive']
    #overallptweets+=len(ptweets)
		sentimentdict = dict()
    # percentage of positive tweets
		sentimentdict['Company'] = company
		sentimentdict['Positive'] = round(100*len(ptweets)/len(tweets[company]),2)
		print("Positive tweets percentage: {} %".format(round(100*len(ptweets)/len(tweets[company]),2)))
    
    # picking negative tweets from tweets
		ntweets = [tweet for tweet in tweets[company] if tweet['sentiment'] == 'negative']
    #overallntweets+=len(ntweets)
    
    # percentage of negative tweets
		sentimentdict['Negative'] = round(100*len(ntweets)/len(tweets[company]),2)
		print("Negative tweets percentage: {} %".format(round(100*len(ntweets)/len(tweets[company]),2)))
    
    # percentage of neutral tweets
		sentimentdict['Neutral'] = round(100*(len(tweets[company]) -(len( ntweets )+len( ptweets)))/len(tweets[company]),2)
		print("Neutral tweets percentage: {} % ".format(round(100*(len(tweets[company]) -(len( ntweets )+len( ptweets)))/len(tweets[company]),2)))
		sentimentlist.append(sentimentdict)
		# printing first 5 positive tweets
		#print("\n\nPositive tweets:")
		#for tweet in ptweets[:10]:
		#   print(tweet['text'])
		
		# printing first 5 negative tweets
		#print("\n\nNegative tweets:")
		#for tweet in ntweets[:10]:
		#  print(tweet['text'])

  #print("Total Positive tweets:",overallptweets)
  #print("Total Negative tweets:",overallntweets)
	if len(overallptweets)>len(overallntweets):
		print("\n\nBullish Market")
	elif len(overallptweets)<len(overallntweets):
		print("\n\nBearish Market")
	else:
		print("\n\nNeutral Market")


if __name__ == "__main__":
	# calling main function
	main()


Market Negative tweets percentage: 14.285714285714286 %
Market Positive tweets percentage: 52.38095238095238 %


Company -  Adani Enterprises
Adani Enterprises
Positive tweets percentage: 29.69 %
Negative tweets percentage: 10.94 %
Neutral tweets percentage: 59.38 % 


Company -  Adani Ports and Special Economic Zone
Adani Ports and Special Economic Zone
Positive tweets percentage: 93.75 %
Negative tweets percentage: 0.0 %
Neutral tweets percentage: 6.25 % 


Company -  Apollo Hospitals Enterprise
Apollo Hospitals Enterprise
Positive tweets percentage: 22.22 %
Negative tweets percentage: 0.0 %
Neutral tweets percentage: 77.78 % 


Company -  Asian Paints
Asian Paints
Positive tweets percentage: 26.09 %
Negative tweets percentage: 21.74 %
Neutral tweets percentage: 52.17 % 


Company -  Axis Bank
Axis Bank
Positive tweets percentage: 8.51 %
Negative tweets percentage: 18.09 %
Neutral tweets percentage: 73.4 % 


Company -  Bajaj Auto
Bajaj Auto
Positive tweets percentage: 29.55 %
Negati

In [6]:
sentimentlist

[{'Company': 'Adani Enterprises',
  'Positive': 29.69,
  'Negative': 10.94,
  'Neutral': 59.38},
 {'Company': 'Adani Ports and Special Economic Zone',
  'Positive': 93.75,
  'Negative': 0.0,
  'Neutral': 6.25},
 {'Company': 'Apollo Hospitals Enterprise',
  'Positive': 22.22,
  'Negative': 0.0,
  'Neutral': 77.78},
 {'Company': 'Asian Paints',
  'Positive': 26.09,
  'Negative': 21.74,
  'Neutral': 52.17},
 {'Company': 'Axis Bank',
  'Positive': 8.51,
  'Negative': 18.09,
  'Neutral': 73.4},
 {'Company': 'Bajaj Auto',
  'Positive': 29.55,
  'Negative': 20.45,
  'Neutral': 50.0},
 {'Company': 'Bajaj Finance',
  'Positive': 24.1,
  'Negative': 12.05,
  'Neutral': 63.86},
 {'Company': 'Bajaj Finserv',
  'Positive': 14.14,
  'Negative': 11.11,
  'Neutral': 74.75},
 {'Company': 'Bharat Petroleum Corporation',
  'Positive': 21.43,
  'Negative': 50.0,
  'Neutral': 28.57},
 {'Company': 'Bharti Airtel',
  'Positive': 42.17,
  'Negative': 15.66,
  'Neutral': 42.17},
 {'Company': 'Britannia Industr

In [ ]:
import json

In [ ]:
companylist = json.dumps(sentimentlist,indent = 2)

In [ ]:
file = open("../newsapp/mynews/src/Companylist.js", "w")
file.write(companylist)
file.close()